## 점진적학습 - 확률적경사하강법

In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import seaborn as sns
import platform

# seaborn 설정 리셋
sns.reset_defaults()

# 폰트설정
if platform.system() == 'Windows' :
    path = 'c:/Windows/Fonts/malgun.ttf'
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
elif platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
else :
    print('Check your OS System')
    
# 그래프에 마이너스 표시
matplotlib.rcParams['axes.unicode_minus'] = False

In [21]:
# <문제> 와인 데이터 사용
# 와인의 화학 조성을 사용하여 와인의 종류 예측
# 특성 이름을 담고있는 key값 = feature_names
# 특성 데이터를 담고 있는 key값 = data
# 와인 종류를 담고있는 key 값 = target_names
    # 범주는 'class_0'과, 'class_1'만 사용 (0과 1로 변경하여 사용)
    # 0 = 레드와인 , 1 = 화이트 와인

## 훈련데이터, 타겟데이터 설정, 분류 및 정제

In [584]:
# 데이터 불러오기
from sklearn.datasets import load_wine
wine_all = load_wine()

In [585]:
# 데이터 전처리 및 가공을 위한 데이터프레임 형태 변경
wine = pd.DataFrame(wine_all['data'], columns= wine_all['feature_names'])

In [586]:
# 타겟데이터 입력
wine['class'] = wine_all['target']

In [587]:
# class_2 제외
wine = wine[wine['class'] != 2]

In [588]:
# 데이터 전처리 - 결측치, 이상치 제거
wine.info()
wine.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 130 entries, 0 to 129
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   alcohol                       130 non-null    float64
 1   malic_acid                    130 non-null    float64
 2   ash                           130 non-null    float64
 3   alcalinity_of_ash             130 non-null    float64
 4   magnesium                     130 non-null    float64
 5   total_phenols                 130 non-null    float64
 6   flavanoids                    130 non-null    float64
 7   nonflavanoid_phenols          130 non-null    float64
 8   proanthocyanins               130 non-null    float64
 9   color_intensity               130 non-null    float64
 10  hue                           130 non-null    float64
 11  od280/od315_of_diluted_wines  130 non-null    float64
 12  proline                       130 non-null    float64
 13  class

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,class
count,130.000000,130.000000,130.000000,130.000000,130.000000,130.000000,130.000000,130.000000,130.000000,130.000000,130.000000,130.000000,130.000000,130.000000
mean,12.944077,1.968077,2.340462,18.785385,99.900000,2.522692,2.490000,0.330231,1.752385,4.194769,1.058892,2.954385,790.092308,0.546154
std,0.888769,0.879968,0.297162,3.400459,15.381379,0.545393,0.737765,0.109087,0.539572,1.625782,0.168690,0.475166,352.513026,0.499791
min,11.030000,0.740000,1.360000,10.600000,70.000000,1.100000,0.570000,0.130000,0.410000,1.280000,0.690000,1.590000,278.000000,0.000000
25%,12.227500,1.510000,2.170000,16.525000,88.000000,2.147500,2.002500,0.260000,1.422500,2.862500,0.940000,2.732500,474.000000,0.000000
50%,13.010000,1.730000,2.320000,18.550000,98.000000,2.560000,2.565000,0.300000,1.705000,3.870000,1.050000,2.980000,716.000000,1.000000
75%,13.727500,2.057500,2.527500,20.950000,108.000000,2.937500,2.997500,0.397500,2.005000,5.375000,1.167500,3.300000,1063.750000,1.000000
max,14.830000,5.800000,3.230000,30.000000,162.000000,3.880000,5.080000,0.660000,3.580000,8.900000,1.710000,4.000000,1680.000000,1.000000


In [589]:
wine

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,class
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,12.07,2.16,2.17,21.0,85.0,2.60,2.65,0.37,1.35,2.76,0.86,3.28,378.0,1
126,12.43,1.53,2.29,21.5,86.0,2.74,3.15,0.39,1.77,3.94,0.69,2.84,352.0,1
127,11.79,2.13,2.78,28.5,92.0,2.13,2.24,0.58,1.76,3.00,0.97,2.44,466.0,1
128,12.37,1.63,2.30,24.5,88.0,2.22,2.45,0.40,1.90,2.12,0.89,2.78,342.0,1


In [590]:
# 상관계수 확인
wine.corr()

# <상관관계 분석>
# alcalinity_of_ash 과 nonflavanoid_phenols 특성이 각각 0.47, 0.34로 양의 상관관계를 보임

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,class
alcohol,1.000000,0.021014,0.190818,-0.461771,0.330544,0.485061,0.543729,-0.301314,0.190869,0.741935,0.026162,0.290491,0.756168,-0.824399
malic_acid,0.021014,1.000000,0.121625,0.141753,-0.016360,0.030038,0.057852,0.055669,0.136454,-0.108202,-0.408645,0.166457,-0.107553,-0.044302
ash,0.190818,0.121625,1.000000,0.369136,0.306593,0.254490,0.377025,0.179396,0.077778,0.253347,0.041164,0.214496,0.304101,-0.354548
alcalinity_of_ash,-0.461771,0.141753,0.369136,1.000000,-0.123546,-0.228873,-0.180189,0.335042,-0.096965,-0.433335,-0.035805,-0.000632,-0.427021,0.470437
magnesium,0.330544,-0.016360,0.306593,-0.123546,1.000000,0.304089,0.255309,-0.212922,0.278018,0.345495,0.071684,0.130838,0.426982,-0.383085
total_phenols,0.485061,0.030038,0.254490,-0.228873,0.304089,1.000000,0.846509,-0.444721,0.444117,0.590428,-0.008302,0.494673,0.506179,-0.532692
flavanoids,0.543729,0.057852,0.377025,-0.180189,0.255309,0.846509,1.000000,-0.358804,0.542889,0.707064,-0.006622,0.537528,0.545781,-0.610730
nonflavanoid_phenols,-0.301314,0.055669,0.179396,0.335042,-0.212922,-0.444721,-0.358804,1.000000,-0.337243,-0.280354,0.051340,-0.468611,-0.330049,0.337489
proanthocyanins,0.190869,0.136454,0.077778,-0.096965,0.278018,0.444117,0.542889,-0.337243,1.000000,0.272754,-0.010739,0.342529,0.274927,-0.249205
color_intensity,0.741935,-0.108202,0.253347,-0.433335,0.345495,0.590428,0.707064,-0.280354,0.272754,1.000000,0.009784,0.208163,0.775315,-0.750613


In [591]:
# 입력데이터, 타겟데이터 분리
wine_input = wine.iloc[:,0:13]
wine_target = wine['class']

In [592]:
wine_input

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,12.07,2.16,2.17,21.0,85.0,2.60,2.65,0.37,1.35,2.76,0.86,3.28,378.0
126,12.43,1.53,2.29,21.5,86.0,2.74,3.15,0.39,1.77,3.94,0.69,2.84,352.0
127,11.79,2.13,2.78,28.5,92.0,2.13,2.24,0.58,1.76,3.00,0.97,2.44,466.0
128,12.37,1.63,2.30,24.5,88.0,2.22,2.45,0.40,1.90,2.12,0.89,2.78,342.0


In [593]:
# 배열형태로 변환
wine_input = wine_input.to_numpy()
wine_target = wine_target.to_numpy()

In [594]:
# 훈련데이터, 테스트데이터 분리
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = \
    train_test_split(wine_input, wine_target, shuffle=wine_target, random_state=4)

In [595]:
# 데이터 분리 확인
print(train_input.shape)
print(train_target.shape)
print(test_input.shape)
print(test_target.shape)

(97, 13)
(97,)
(33, 13)
(33,)


In [596]:
# 정규화
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train_input, train_target)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

## 예측데이터 설정 및 정제

In [597]:
# 기존의 데이터를 예측데이터로 가정
wine_pred = wine

In [598]:
# 예측데이터 정제 - 타겟 데이터 제거
wine_pred_input = wine_pred.iloc[:,0:13]

In [599]:
# 예측데이터 배열형태로 변환
wine_pred_input = wine_pred_input.to_numpy()

In [600]:
# 예측데이터 정규화
wine_pred_scaled = ss.transform(wine_pred_input)

## 1. KNN 모델 

In [601]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(train_scaled, train_target)
print('KNN   훈련데이터 정확도 :', round(knn.score(train_scaled, train_target),4))
print('KNN 테스트데이터 정확도 :', round(knn.score(test_scaled, test_target),4))

# <해석>
# random_state(랜덤 시드값) : 4
# n_neighbors(이웃의 수) : 5
# 훈련데이터 정확도 : 0.9794
# 테스트데이터 정확도 : 0.9697
# 모든 특성(퓨처)를 사용하였고 편향을 최대한 줄이기위해 랜덤 시드값을 사용함(4로 설정)
# 전체적인 정확도가 우수하나 약간의 훈련데이터 과대적합이 의심됨

KNN   훈련데이터 정확도 : 0.9794
KNN 테스트데이터 정확도 : 0.9697


In [501]:
# 예측데이터 입력 후 결과 데이터프레임에 추가
wine_pred['KNN_class_pred'] = knn.predict(wine_pred_scaled)
wine_pred[['class','KNN_class_pred']]

,class,KNN_class_pred
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
125,1,1
126,1,1
127,1,1
128,1,1


In [502]:
# 정답 갯수, 오답갯수 확인
# 정답갯수
knn_y_cnt= len(wine_pred[wine_pred['class'] == wine_pred['KNN_class_pred']])
# 오답갯수
knn_n_cnt = len(wine_pred[wine_pred['class'] != wine_pred['KNN_class_pred']])
# 전체갯수
knn_all_cnt = knn_y_cnt + knn_n_cnt

# -----------------------------------------------------------------------------------------
# 정답률, 오답률 확인
# 정답률
knn_y_per = round(knn_y_cnt / knn_all_cnt * 100 , 2)
# 오답률
knn_n_per = round(knn_n_cnt / knn_all_cnt * 100 , 2)
# 전체확률(100%)
knn_all_per = knn_y_per + knn_n_per

print('----------------------------------------------------------------------------------')
print('<KNN 모델 정답확인>')
print('총 갯수 [{}]건 중에, 정답갯수[{}]건, 오답갯수[{}]건'.format(knn_all_cnt, knn_y_cnt, knn_n_cnt))
print('----------------------------------------------------------------------------------')
print('<KNN 모델 정답률>')
print('총 [{}%] 중에, 정답률[{}%], 오답률[{}%]'.format(knn_all_per, knn_y_per, knn_n_per))
print('----------------------------------------------------------------------------------')

----------------------------------------------------------------------------------
<KNN 모델 정답확인>
총 갯수 [130]건 중에, 정답갯수[127]건, 오답갯수[3]건
----------------------------------------------------------------------------------
<KNN 모델 정답률>
총 [100.0%] 중에, 정답률[97.69%], 오답률[2.31%]
----------------------------------------------------------------------------------


In [503]:
# <최종 분석>
# 모델의 정확도가 우수하여 97.69%의 확률로 정확한 예측을 할 수 있다고 판단됨  

## 2. 로지스틱 회귀분류 모델

In [506]:
from sklearn.linear_model import LogisticRegression

# 최적의 C값(가중치) 찾아내기
for c in np.arange(0.001, 1, 0.001) :
    lr = LogisticRegression(C=c, max_iter=1000)
    lr.fit(train_scaled, train_target)
    if (lr.score(train_scaled, train_target) < 1) & (lr.score(test_scaled, test_target) > 0.93) :
            print('C값(가중치) :', c)
            print('로지스틱   훈련데이터 정확도 :', round(lr.score(train_scaled, train_target),4))
            print('로지스틱 테스트데이터 정확도 :', round(lr.score(test_scaled, test_target),4))
            break
            
# <해석>
# random_state(랜덤 시드값) : 4
# C값(가중치) : 0.029
# max_iter(훈련횟수) : 1000
# 훈련데이터 정확도 : 0.9897
# 테스트데이터 정확도 : 0.9394
# 모든 특성(퓨처)를 사용하였고 편향을 최대한 줄이기위해 랜덤 시드값을 사용함(4로 설정)
# 반복문을 통한 최적의 C값(가중치) 탐색 및 설정
# 전체적인 정확도가 우수하나 훈련데이터 과대적합이 의심됨

C값(가중치) : 0.029
로지스틱   훈련데이터 정확도 : 0.9897
로지스틱 테스트데이터 정확도 : 0.9394


In [507]:
# 예측데이터 입력 후 결과 데이터프레임에 추가
wine_pred['LR_class_pred'] = lr.predict(wine_pred_scaled)
wine_pred[['class','LR_class_pred']]

,class,LR_class_pred
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
125,1,1
126,1,1
127,1,1
128,1,1


In [508]:
# 정답 갯수, 오답갯수 확인
# 정답갯수
lr_y_cnt= len(wine_pred[wine_pred['class'] == wine_pred['LR_class_pred']])
# 오답갯수
lr_n_cnt = len(wine_pred[wine_pred['class'] != wine_pred['LR_class_pred']])
# 전체갯수
lr_all_cnt = lr_y_cnt + lr_n_cnt

# -----------------------------------------------------------------------------------------
# 정답률, 오답률 확인
# 정답률
lr_y_per = round(lr_y_cnt / lr_all_cnt * 100 , 2)
# 오답률
lr_n_per = round(lr_n_cnt / lr_all_cnt * 100 , 2)
# 전체확률(100%)
lr_all_per = lr_y_per + lr_n_per

print('----------------------------------------------------------------------------------')
print('<로지스틱 회귀 분류 모델 정답확인>')
print('총 갯수 [{}]건 중에, 정답갯수[{}]건, 오답갯수[{}]건'.format(lr_all_cnt, lr_y_cnt, lr_n_cnt))
print('----------------------------------------------------------------------------------')
print('<로지스틱 회귀 분류 모델 정답률>')
print('총 [{}%] 중에, 정답률[{}%], 오답률[{}%]'.format(lr_all_per, lr_y_per, lr_n_per))
print('----------------------------------------------------------------------------------')

----------------------------------------------------------------------------------
<로지스틱 회귀 분류 모델 정답확인>
총 갯수 [130]건 중에, 정답갯수[127]건, 오답갯수[3]건
----------------------------------------------------------------------------------
<로지스틱 회귀 분류 모델 정답률>
총 [100.0%] 중에, 정답률[97.69%], 오답률[2.31%]
----------------------------------------------------------------------------------


In [509]:
# <최종 분석>
# 모델의 정확도가 우수하여 97.69%의 확률로 정확한 예측을 할 수 있다고 판단됨  

## 3. SGD 모델


In [583]:
from sklearn.linear_model import SGDClassifier

# 최적의 max_iter 값 찾아내기
sc = SGDClassifier(max_iter= 10, random_state=4)
sc.fit(train_scaled, train_target)
print('SGD 훈련데이터 정확도 :', round(sc.score(train_scaled, train_target),4))
print('SGD 테스트데이터 정확도 :', round(sc.score(test_scaled, test_target),4))

# <해석>
# random_state(랜덤 시드값) : 4
# max_iter(훈련횟수) : 10
# 훈련데이터 정확도 : 0.9897
# 테스트데이터 정확도 : 0.9697
# 모든 특성(퓨처)를 사용하였고 편향을 최대한 줄이기위해 랜덤 시드값을 사용함(4로 설정)
# 전체적인 정확도가 매우 우수하나 훈련데이터 과대적합이 의심됨

SGD 훈련데이터 정확도 : 0.9897
SGD 테스트데이터 정확도 : 0.9697


In [544]:
# 예측데이터 입력 후 결과 데이터프레임에 추가
wine_pred['SGD_class_pred'] = sc.predict(wine_pred_scaled)
wine_pred[['class','SGD_class_pred']]

,class,SGD_class_pred
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
125,1,1
126,1,1
127,1,1
128,1,1


In [582]:
# 정답 갯수, 오답갯수 확인
# 정답갯수
sc_y_cnt= len(wine_pred[wine_pred['class'] == wine_pred['SGD_class_pred']])
# 오답갯수
sc_n_cnt = len(wine_pred[wine_pred['class'] != wine_pred['SGD_class_pred']])
# 전체갯수
sc_all_cnt = sc_y_cnt + sc_n_cnt

# -----------------------------------------------------------------------------------------
# 정답률, 오답률 확인
# 정답률
sc_y_per = round(sc_y_cnt / sc_all_cnt * 100 , 2)
# 오답률
sc_n_per = round(sc_n_cnt / sc_all_cnt * 100 , 2)
# 전체확률(100%)
sc_all_per = sc_y_per + sc_n_per

print('----------------------------------------------------------------------------------')
print('<SGD 모델 정답확인>')
print('총 갯수 [{}]건 중에, 정답갯수[{}]건, 오답갯수[{}]건'.format(sc_all_cnt, sc_y_cnt, sc_n_cnt))
print('----------------------------------------------------------------------------------')
print('<SGD 모델 정답률>')
print('총 [{}%] 중에, 정답률[{}%], 오답률[{}%]'.format(sc_all_per, sc_y_per, sc_n_per))
print('----------------------------------------------------------------------------------')

----------------------------------------------------------------------------------
<SGD 모델 정답확인>
총 갯수 [130]건 중에, 정답갯수[128]건, 오답갯수[2]건
----------------------------------------------------------------------------------
<SGD 모델 정답률>
총 [100.0%] 중에, 정답률[98.46%], 오답률[1.54%]
----------------------------------------------------------------------------------


In [ ]:
# <최종 분석>
# 모델의 정확도가 매우 우수하여 98.46% 의 확률로 정확한 예측을 할 수 있다고 판단됨  

## 4. 결정트리 모델

In [580]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree

dt = DecisionTreeClassifier(random_state=4, max_depth=1)
dt.fit(train_scaled, train_target)
print('결정트리   훈련데이터 정확도 :', round(dt.score(train_scaled, train_target),4))
print('결정트리 테스트데이터 정확도 :', round(dt.score(test_scaled, test_target),4))

# <해석>
# random_state(랜덤 시드값) : 4
# max_depth(가지치기횟수) : 1
# 훈련데이터 정확도 : 0.9485
# 테스트데이터 정확도 : 0.9394
# 모든 특성(퓨처)를 사용하였고 편향을 최대한 줄이기위해 랜덤 시드값을 사용함(4로 설정)
# 전체적인 정확도가 좋으나 훈련데이터 과대적합이 의심됨
# 가지치기 횟수를 2 이상할 경우 훈련데이터의 정확도가 1이 되어 최적의 모델이라 판단하기 어려움(선정불가)

결정트리   훈련데이터 정확도 : 0.9485
결정트리 테스트데이터 정확도 : 0.9394


In [581]:
# 트리 훈련에 사용된 독립변수(특성=퓨처)들의 정확도 확인
    # - 정확도가 가장 높은 독립변수가 훈련에 영향을 가장 많이 미쳤다고 판단
print(dt.feature_importances_)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


## 최종 모델선정 - SGD 모델
### 모델선정이유
- 훈련데이터 정확도 : 0.9897 (98.97%)
- 테스트데이터 정확도 : 0.9697 (96.97%)
- 전체적인 정확도가 매우 우수함 

#### 실제 예측데이터 입력 및 테스트 결과
- 총 갯수 [130]건 중에, 정답갯수[128]건, 오답갯수[2]건
- 총 [100.0%] 중에, 정답률[98.46%], 오답률[1.54%]

- 130개의 예측데이터 입력시 98.46%의 뛰어난 정답률을 확인 할 수 있었음